<a href="https://colab.research.google.com/github/kelseylieberman/Random_Assignments/blob/master/Markov_Decision_Processes_and_Hidden_Markov_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# CSCI 3202, Spring 2020:  Assignment 5
### Due:  Friday 17 April 2020 by 11:59 PM

### Your name: Kelsey Lieberman

---



---
## Problem 1: Navigating an awkward situation with grace and poise

<img src='https://www.explainxkcd.com/wiki/images/5/5f/interaction.png' style="width: 600px;"/>


Suppose you are at a social event where you would like to avoid any interaction with a large number of the other attendees. It's not that you don't like them, it's just that you don't like *talking to* them. A few of your good friends are also in attendance, but they are tucked away in a corner. The rectangular room in which the event is being held spans gridcells at $x=1,2,\ldots, 6$ and $y=1,2,\ldots, 5$. At the eastern edge ($x=6$) of this first floor room, there is a balcony, with a 6-foot drop. If the event becomes unbearably awkward, you can jump off the balcony and run away. Of course, this might hurt a little bit, so we should incorporate this into our reward structure.

The terminal states and rewards associated with them are given in the diagram below. The states are represented as $(x,y)$ tuples. The available actions in non-terminal states include moving exactly 1 unit North (+y), South (-y), East (+x) or West (-x), although you should not include walking into walls, because that would be embarrassing in front of all these other people. Represent actions as one of 'N', 'S', 'E', or 'W'. For now, assume all non-terminal states have a default reward of -0.01, and use a discount factor of 0.99.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/hw06_mdp.png" style="width: 400px;"/>

Use the following transition model for this decision process, if you are trying to move from state $s$ to state $s'$:
* you successfully move from $s$ to $s'$ with probability 0.6
* the remaining 0.4 probability is spread equally likely across state $s$ **and** all adjacent (N/S/E/W) states except for $s'$. Note that this does not necessarily mean that all adjacent states have 0.1, because some states do not have 4 adjacent states.


### (1a)

Complete the `MDP` class below. The docstring comments provide some desired specifications. You may add additional methods or attributes, if you would like.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import unittest

In [2]:
class MDP:
    def __init__(self, nrow, ncol, terminal, default_reward, discount):
        '''Create/store the following attributes:
        states -- list of all the states (x,y) tuples
        terminal_states -- is a dictionary with terminal state keys, and rewards as values
        default_reward -- is the reward for being in any non-terminal state
        df -- discount factor
        ... and anything else you decide will be useful!
        '''
        self.states = []
        for x in range(ncol):
            for y in range(nrow):
                self.states.append((x+1, y+1))
        self.terminal_states = terminal
        self.default_reward = default_reward
        self.df = discount
        
        

    def actions(self, state):
        '''Return a list of available actions from the given state.
        [None] are the actions available from a terminal state.
        '''
        
        if state in self.terminal_states:
            return [None]
        else:
            #corners
            if state == (1, 1):
                return(['N', 'E'])
            elif state == (1, nrow):
                return(['S', 'E'])
            elif state == (ncol, 1):
                return(['N', 'W'])
            elif state == (ncol, nrow):
                return(['S', 'W'])
            #edges
            elif state[0] == 1:
                return ['N', 'S', 'E']
            elif state[0] == ncol:
                return ['S', 'N', 'W']
            elif state[1] == 1:
                return ['N', 'W', 'E']
            elif state[1] == nrow:
                return ['S', 'E', 'W']
            #anything else
            else:
                return ['N', 'S', 'W', 'E']
                
        
        
        
    def reward(self, state):
        '''Return the reward for being in the given state'''
        
        if state in self.terminal_states:
            return(self.terminal_states[state])
        else:
            return self.default_reward       
        
    def result(self, state, action):
        '''Return the resulting state (as a tuple) from doing the given
        action in the given state, without uncertainty. Uncertainty
        is incorporated into the transition method.
        state -- a tuple representing the current state
        action -- one of N, S, E or W, as a string
        '''
        
        s = list(state).copy()
        if action == 'N':
            s[1] += 1
        elif action == 'E':
            s[0] += 1
        elif action == 'S':
            s[1] -= 1
        elif action == 'W':
            s[0] -= 1
        else:
            return "invalid move"
        return tuple(s)
        
        
                
    def transition(self, state, action):
        '''Return the probabilities and subsequent states associated
        with taking the given action from the given state. Can be done
        however you want, so that it works with your value/policy iteration.
        '''
        if action is None:
            return [(0, state)]
        tp = []
        if action in self.actions(state):
            prob = 0.4/(len(self.actions(state)))
            for item in self.actions(state):
                if item == action:
                    tp.append((0.6, self.result(state, item)))
                else:
                    tp.append((prob, self.result(state, item)))
            tp.append((prob, state))
        else:
            prob = 1/len(self.actions(state)+1)
            for item in self.actions(state):
                tp.append((prob, self.result(state, item)))
            tp.append((prob, state))
        return tp

**Now:** create an `MDP` object to represent the decision process in this problem.

To test and get comfortable with your `MDP` class methods and attributes, and making the relevant calculations with this structure, calculate the expected utility of walking north from (1,1). Assume initially that all states $(x,y)$ have a utility of $x+y$ (including the terminal states).

In [3]:
nrow = 5
ncol = 6
terminal = {(1, 5): 2, (3, 5): 1, (6, 5): -5, (1, 4): 2, (3, 4): -1, (4, 4): -1,
            (6, 4): -5, (1, 3): -1,(4, 3): -1, (6, 3): -5, (6,2): -5, (2, 1): -1,
            (3,1): -1, (6,1): -5}
default_reward = -0.01
discount = 0.99

Markoff = MDP(nrow, ncol, terminal, default_reward, discount)

def north_expected_utility(mdp):
    s = [(1, 2),(1, 3),(1, 4),(1, 5)]
    prob = 0.6
    expectation = discount*sum([prob*mdp.reward(move) for move in s])
    return expectation
print(north_expected_utility(Markoff))
        

1.77606


#### Unit tests

Test your MDP with a unit test. This is for you, so you can make it as exaustive or inexhaustive as you please

In [4]:
class Tests_Problem1(unittest.TestCase):
    def setUp(self):
        nrow = 3
        ncol = 3
        default_reward = -0.2
        discount = 0.5
        terminal = {(1,3):-1, (1,2):2}
        self.mdp = MDP(nrow, ncol, terminal, default_reward, discount)
    def test_actions_some(self):
        self.assertEqual(set(self.mdp.actions((2,2))) == {'N','S','E','W'}, True)
    def test_actions_few(self):
        self.assertEqual(set(self.mdp.actions((1,1))) == {'N','E'}, True)
    def test_actions_none(self):
        self.assertEqual(set(self.mdp.actions((1,2))) == {None}, True)
    def test_reward_t(self):
        self.assertEqual(self.mdp.reward((1,2)) == 2, True)
    def test_reward_nt(self):
        self.assertEqual(self.mdp.reward((2,2)) == -0.2, True)
    def test_result_nt(self):
        self.assertEqual(self.mdp.result((1,1), 'N') == (1,2), True)

In [5]:
tests_to_run = unittest.TestLoader().loadTestsFromModule(Tests_Problem1())
unittest.TextTestRunner().run(tests_to_run)

......
----------------------------------------------------------------------
Ran 6 tests in 0.007s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

### (1b)

Implement value iteration to calculate the utilities for each state.  Also implement a function that takes as arguments an `MDP` object and a dictionary of state-utility pairs (key-value) and returns a dictionary for the optimal policy.  The optimal policy dictionary should have state tuples as keys and the optimal move (None, N, S, E or W) as values.

In [6]:
def value_iteration(mdp, tol=1e-3):
    df = 1-tol

    # initilize utility for all states
    utility_new = {s : s[0]+s[1] for s in mdp.states}

    # iterate:
    while True:

        # make a copy of current utility estimate, to be modified
        utility_old = utility_new.copy()

        # initialize maximum change to 0
        max_change = 0

        # for each state s:
        for s in mdp.states:

            # for each available action, what next states
            # are possible, and their probabilities?
            next_states = [mdp.transition(s, a) for a in mdp.actions(s)]

            # calculate the maximum expected utility
            best_utility = float("-inf")
            for k in range(len(next_states)):
                newsum = sum([next_states[k][j][0]*utility_old[next_states[k][j][1]] for j in range(len(next_states[k]))])
                best_utility = max(best_utility, newsum)
                if len(next_states)==1:
                    best_utility = newsum

            # new utility of s = reward(s) + 
            #                    discounted max expected utility
            utility_new[s] = mdp.reward(s) + df*best_utility

            # update maximum change in utilities, if needed
            max_change = max(max_change, abs(utility_new[s]-utility_old[s]))

        # if maximum change in utility from one iteration to the
        # next is less than some tolerance, break!
        if (df==1 and max_change < tol) or max_change < tol*(1-df)/df:
            break 
    return utility_new
        
def find_policy(mdp, utility):
    # initialize the policy for each state
    policy = {s : None for s in mdp.states}

    # loop over states to find the action that maximizes expected utility
    for s in mdp.states:

        # initialize the best utility to something very bad, so we can improve it
        best_utility = (float("-inf"), None)

        # loop over actions, find which gives the highest expected utility
        for a in mdp.actions(s):

            # calculate the expected utility of action a from state s
            newsum = sum([p*utility[s2] for p, s2 in mdp.transition(s,a)])

            # if this action has higher expected utility than the current best,
            # replace the best (utility, action) tuple with this one
            if newsum > best_utility[0]:
                best_utility = (newsum, a)

        # now we have the action (second element) that leads
        # to the highest expected utility (first element)
        policy[s] = best_utility[1]

    # upon exit, policy has the optimal policy for each state
    return policy
    
    

Now actually use your `value_iteration` and `find_policy` functions to calculate the utility for each state in this MDP, and the optimal action in each state.

As a sanity check, print the utilities of these terminal states:
1. `utility[(1,5)]`
1. `utility[(6,1)]`

and print the utility of these states that are nearby to terminal states, so their utilities should be similar to the nearby terminal states' utilities:
1. `utility[(2,5)]`
1. `utility[(5,3)]`

And print the policy for these states to make sure they make sense:
1. `policy[(2,4)]`
1. `policy[(1,1)]`

In [7]:
U = value_iteration(Markoff)
P = find_policy(Markoff, U)
print(U[(1, 5)], U[(6, 1)])
print(U[(2, 5)], U[(5, 3)])
print(P[(2,4)], P[(1,1)])

2.0 -5.0
1.7582498999401663 -1.4037475029624822
W N


### (1c)

If we enter the room at (5,1), what is the optimal path for us to follow?  Create a graphic to illustrate this policy pathway, either by generating a plot in Python (like the maze solution path) or by uploading a hand-drawn image and including it below.

In [8]:
U = value_iteration(Markoff)
P = find_policy(Markoff, U)
current_state = (5, 1)
moves = {}
while Markoff.actions(current_state) != [None]:
    print(current_state, P[current_state])
    moves[current_state] = P[current_state]
    current_state = Markoff.result(current_state, P[current_state])
print(current_state, U[current_state])
print()

#grid/board display
for y in range(nrow, 0, -1):
    for x in range(1, ncol+1):
        if (x, y) in moves:
            print('[ ' + str(moves[(x, y)]) + ' ]', end = '')
        elif (x, y) in Markoff.terminal_states:
            if Markoff.terminal_states[(x, y)] < 0:
                print('[' + str(Markoff.terminal_states[(x, y)]) + ' ]', end = '')
            else:
                print('[ ' + str(Markoff.terminal_states[(x, y)]) + ' ]', end = '')
        else:
            print('[   ]', end = '')
    print()

(5, 1) W
(4, 1) N
(4, 2) W
(3, 2) W
(2, 2) N
(2, 3) N
(2, 4) W
(1, 4) 2.0

[ 2 ][   ][ 1 ][   ][   ][-5 ]
[ 2 ][ W ][-1 ][-1 ][   ][-5 ]
[-1 ][ N ][   ][-1 ][   ][-5 ]
[   ][ N ][ W ][ W ][   ][-5 ]
[   ][-1 ][-1 ][ N ][ W ][-5 ]


<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/hw06_mdp_path.png" style="width: 300px;"/>

### (1d)

From (3,2) the optimal move is to walk West. If we are trying to go talk to our friends in the Northwest corner, why would we rather do this than walk North first, then West?

It is better to walk West than North in this case because walking North first results in a 0.2 chance that we accidentally end up at one of the -1 terminal states to the north or east of our position (this is taking into consideration only the current state at (3, 3) and that our next intentional move is west. Moving west first is better because there is only a 0.1 chance that we accidentally end up in a -1 terminal state (given our next move would be north after (2, 2)) So the expected utility of moving to the west from (3, 2) is higher than the expected utility of moving north from the same position. 

### (1e)

How painfully awkward do you need to set the default reward for non-terminal states before the optimal move from (5,1) becomes jumping off the balcony immediately and running away?  Round your answer to two decimal places.

In [9]:
nrow = 5
ncol = 6
terminal = {(1, 5): 2, (3, 5): 1, (6, 5): -5, (1, 4): 2, (3, 4): -1, (4, 4): -1,
            (6, 4): -5, (1, 3): -1,(4, 3): -1, (6, 3): -5, (6,2): -5, (2, 1): -1,
            (3,1): -1, (6,1): -5}
default_reward = -2.06
discount = 0.99

Awk_model = MDP(nrow, ncol, terminal, default_reward, discount)

U = value_iteration(Awk_model)
P = find_policy(Awk_model, U)
current_state = (5, 1)
moves = {}
while Awk_model.actions(current_state) != [None]:
    print(current_state, P[current_state])
    moves[current_state] = P[current_state]
    current_state = Awk_model.result(current_state, P[current_state])
print(current_state, U[current_state])
print()

#grid/board display
for y in range(nrow, 0, -1):
    for x in range(1, ncol+1):
        if (x, y) in moves:
            print('[ ' + str(moves[(x, y)]) + '   ]', end = '')
        elif (x, y) in Awk_model.terminal_states:
            if Awk_model.terminal_states[(x, y)] < 0:
                print('[' + str(Awk_model.terminal_states[(x, y)]) + '   ]', end = '')
            else:
                print('[ ' + str(Awk_model.terminal_states[(x, y)]) + '   ]', end = '')
        else:
            if round(U[(x, y)], 2) < 0:
                print('[' + str("{:.2f}".format(round(U[(x, y)], 2))) + ']', end = '')
            else:
                print('[' + str("{:.2f}".format(round(U[(x, y)], 2))) + ' ]', end = '')
    print()
print('\n -2.06 is the lowest default reward that results in an optimal move of jumping off the balcony, '
      + 'AKA going east from (5,1). This was found through trial and error. This assumes the discount '
     + 'factor and other parameters for the model remain the same as for the previous parts of problem 1')

(5, 1) E
(6, 1) -5.0

[ 2   ][-1.09][ 1   ][-2.72][-5.75][-5   ]
[ 2   ][-1.64][-1   ][-1   ][-4.69][-5   ]
[-1   ][-4.07][-4.01][-1   ][-4.86][-5   ]
[-4.42][-4.39][-4.42][-4.83][-7.46][-5   ]
[-4.43][-1   ][-1   ][-5.00][ E   ][-5   ]

 -2.06 is the lowest default reward that results in an optimal move of jumping off the balcony, AKA going east from (5,1). This was found through trial and error. This assumes the discount factor and other parameters for the model remain the same as for the previous parts of problem 1


### (1f)

In **1e** we assumed a certain level of loss (negative reward) just for being present.  But a more realistic approach might be to instead change the reward structure for the terminal states. Consider the terminal states with -1 reward in the default model. Let $R^*$ denote the reward associated with these states. How low does $R^*$ need to be in order for us to immediately jump off the balcony and run away? Use the default non-terminal state reward of -0.01. Write a few sentences interpreting your result.

In [16]:
nrow = 5
ncol = 6
R_star = -10.78
terminal = {(1, 5): 2, (3, 5): 1, (6, 5): -5, (1, 4): 2, (3, 4): R_star, (4, 4): R_star,
            (6, 4): -5, (1, 3): R_star,(4, 3): R_star, (6, 3): -5, (6,2): -5, (2, 1): R_star,
            (3,1): R_star, (6,1): -5}
default_reward = -0.01
discount = 0.99

Awk_model = MDP(nrow, ncol, terminal, default_reward, discount)

U = value_iteration(Awk_model)
P = find_policy(Awk_model, U)
current_state = (5, 1)
moves = {}
while Awk_model.actions(current_state) != [None]:
    print(current_state, P[current_state])
    moves[current_state] = P[current_state]
    current_state = Awk_model.result(current_state, P[current_state])
print(current_state, U[current_state])
print()

#grid/board display
for y in range(nrow, 0, -1):
    for x in range(1, ncol+1):
        if (x, y) in moves:
            print('[ ' + str(moves[(x, y)]) + '   ]', end = '')
        elif (x, y) in Awk_model.terminal_states:
            if Awk_model.terminal_states[(x, y)] < 0:
                print('[' + str(Awk_model.terminal_states[(x, y)]) + '   ]', end = '')
            else:
                print('[ ' + str(Awk_model.terminal_states[(x, y)]) + '   ]', end = '')
        else:
            if round(U[(x, y)], 2) < 0:
                print('[' + str("{:.2f}".format(round(U[(x, y)], 2))) + ']', end = '')
            else:
                print('[' + str("{:.2f}".format(round(U[(x, y)], 2))) + ' ]', end = '')
    print()
    
print('\n -10.78 is the lowest R* that results in an optimal move of jumping off the balcony, '
      + 'AKA going east from (5,1). This was found through trial and error. This assumes a default '
     + 'reward of -0.01 and other parameters for the model remain the same as for the previous' 
      + ' parts of problem 1')

(5, 1) E
(6, 1) -5.0

[ 2   ][1.54 ][ 1   ][-1.33][-2.29][-5   ]
[ 2   ][0.07 ][-10.78   ][-10.78   ][-3.82][-5   ]
[-10.78   ][-2.05][-4.30][-10.78   ][-4.86][-5   ]
[-5.23][-3.68][-4.76][-5.61][-5.00][-5   ]
[-6.62][-10.78   ][-10.78   ][-6.11][ E   ][-5   ]

 -10.78 is the lowest R* that results in an optimal move of jumping off the balcony, AKA going east from (5,1). This was found through trial and error. This assumes a default reward of -0.01 and other parameters for the model remain the same as for the previous parts of problem 1


### (1g)

Given the problem context, write a few sentences about why this is or is not an appropriate transition model. Include an interpretation of the terminal states.

I think this transition model it not perfect in the context of this problem because people know how to walk; there is not an equal probability I will move in available various directions besides the intended direction. A better model would have higher probabilities associated with accidentally moving too close to a terminal '-1' node because if that represents a person I don't want to talk to, there is a chance I will accidentally get sucked into a conversation with them just by walking near them. Also, there should be a zero probability of staying in the same spot if not in a terminal node, because it would be weird to stand alone for too long a period of time. Also going back and forth seems odd and highly unlikely. The model from the lecture on April 1 seems more likely, where you will either move in the intended direction or to the left or right of the intended direction with lesser probability, because maybe in this scenario I have had a couple drinks and my feet don't always land where I want.

A six foot drop from the balcony doesn't seem so bad, especially if my social anxiety is as described in the problem, so I think -5 consequence is a little extreme considering the situation. The placement of the terminal nodes seems appropriate; clearly the positive terminal nodes are in the corner because the problem says my friends are tucked away in the corner. Also, maybe it would make sense to decrease the penalty of negative terminal nodes adjacent to my friends because at least they may be able to save me from the conversation with a stranger. The terminal nodes at (2,1) and (3,1), for instance, would be worse because they are far away from my friends who would not be able to save me from the conversation at that distance.

<br>

---

## Problem 2: HMMs

You are trying to diagnose whether your computer is broken or not. On a given day, your computer's hidden state is either *broken* or *working*. Each day you make one of the following observations: *blue-screen*, *slow*, or *snappy*, depending on the state of your computer. You decide to use the following HMM to model your daily observations. Note, "Emission Distribution" is another way to descibe the "Sensor Distribution".

<img src="https://i.ibb.co/cNqvdnB/HMM.png" />;




### (2a)
What is the posterior distribution of $X_1$, your computer's state on day one, given the observation (*slow*) on day 1? In other words, find $P(X_1 | E_1 = \textit{slow})$.

$$P(E_1 = slow \mid X_1 = working) = 0.2$$  
$$P(X_1 = working) = 0.9$$  
$P(E_1 = slow) = P(E_1 = slow \mid X_1 = working)P(X_1 = working) + P(E_1 = slow \mid X_1 = broken)P(X_1 = broken)$  
$= (0.2)(0.9) + (0.4)(0.1)$  
$$P(E_1 = slow) = 0.22$$  
$P(X_1 = working | E_1 = slow) = \frac{P(E_1 = slow \mid X_1 = working)P(X_1 = working)}{P(E_1 = slow)}$  
$P(X_1 = working | E_1 = slow) = \frac{(0.2)(0.9)}{(0.22)}$  
$$P(X_1 = working | E_1 = slow) = 0.818$$  
$P(X_1 = broken | E_1 = slow) =  1-P(X_1 = working | E_1 = slow)$  
$$P(X_1 = broken | E_1 = slow) = 0.182$$

### (2b)
What is the posterior distribution of $X_2$, your computer's state on day two, given the observation sequence (*slow*, *slow*)?

Using the Forward Algorithm:  

$P(X_2 = working | E_{1,2}) = P(X_1 = working | E_1 = slow)P(X_2 = working | X_1 = working)P(E_2 = slow | X_2 = working) + P(X_1 = broken | E_1 = slow)P(X_2 = working | X_1 = broken)P(E_2 = slow | X_2 = working)\alpha$  
$ = (0.818)(0.9)(0.2) + (0.182)(0)(0.2)\alpha = 0.147\alpha$  
$$P(X_2 = working | E_{1,2}) = 0.147\alpha$$  

$P(X_2 = broken | E_{1,2}) = P(X_1 = working | E_1 = slow)P(X_2 = broken | X_1 = working)P(E_2 = slow | X_2 = broken) + P(X_1 = broken | E_1 = slow)P(X_2 = broken | X_1 = broken)P(E_2 = slow | X_2 = broken)\alpha$  
$ = (0.818)(0.1)(0.4) + (0.182)(1)(0.4)\alpha = 0.105\alpha$  
$$P(X_2 = broken | E_{1,2}) = 0.105\alpha$$  
  
$P(X_2 = working | E_{1,2}) + P(X_2 = broken | E_{1,2})$  
$0.105\alpha + 0.147\alpha = 1$
$\alpha = 1/(0.105 + 0.147)$  
$$\alpha = 3.968$$  
  
$$P(X_2 = working | E_{1,2}) = 0.147(3.968)$$  
$$=0.583$$  
  
$$P(X_2 = broken | E_{1,2}) = 0.105(3.968)$$  
$$=0.417$$  

<br>

---

## Problem 3: your very own MDP

For this problem, you do not need to write any code, but rather communicate your ideas clearly using complete sentences and descriptions of the concepts the questions ask about. You can, of course, include some pseudocode if it helps, but that is not strictly necessary.

### (3a)

Describe something you think would be interesting to model using a Markov decision process.  Be **creative** - do not use any examples from your homework, class, or the textbook, and if you are working with other students, please **come up with your own example**. There are so, SO many possible answers!

In spirit of the COVID-19 outbreak, I have decided that it would be interesting to model a disease using a Markov decision process. I think this is an excellent use of the Markov decision model and very applicable to this topic in general. The way a virus travels from person to person and how the probability of infection or showing symptoms allows for various states and transition probabilities makes it interesting to think about. In this case, I am imagining that the virus is being modeled by the Markov decision process in the sense that despite its adversarial nature, the model is produced in favor of the virus succeeding in propegating through the world population.

### (3b)

What are the states associated with your MDP?

The states associated with my MDP would be represented by people. The strange thing about this MDP is that it would have to incorporate the inevitable possibility of occupying multiple, even many, states at once. I think this would be relatively easy to model, because each time the virus infected a new person, it would inherit the properties of the first virus in addition to adding an occupied state to the list of states. Ignoring computing power as a factor for the moment, states would be ideally represented as people, one state for each person. The "board" would be the world, and the "board state" would be the number of infected people. Each state would be defined by a primary key or unique identifier and location on the planet. States would also have temporal constraints such that if they have been infected for a long time they either die or recover. When states are a certain radius from other states, they have a probability of infecting the other state (person) which is defined by the distance between the two people. The larger the distance, the lower the probability of infection. Additionally, states could have boolean symptom properties which, when activated change their likelihood of infecting others or dying. Terminal states would be those who die or those who recover.

### (3c)

What is the reward structure associated with your MDP?  Include a discussion of terminal/non-terminal states.

I apologize for the morbid nature of my MDP choice but I'm too deep to turn back now. The reward structure is a little tricky. The default reward would have to be based on time. From the virus's point of view, it wants to infect the entire population of the world. For simplicity let's assume the world population is static, in that no new people are born; the population of the world only decreases from the start of the virus. We need to consider that if the virus takes too long, people will die of old age and other causes, so there needs to be a time-motivating factor for the virus. We can make the default reward per second of the virus -0.0001 per second divided by total number of viruses. This means that the virus is not penalized for creating more viruses and infecting more people, but it is penalized for taking a long time. Each time a new person is infected by the virus, the virus is rewarded with a point. This ensures the virus will try to maximize it's decision process by infecting as many people as possible. If a person recovers from the virus, the virus loses a point. Now, if the virus infects somebody and that person dies without infecting anyone else, the virus loses 0.5 points. This is to ensure that the virus will try to infect people who maximize the infection of others. However, they still gain 0.5 points from the initial infection, so it does not totally discourage infecting everyone on the planet. In summary, we have -0.0001 loss per second / total number of infections as the default reward. +1.0 is rewarded for each new state reached (or person infected). -1.0 for each person who recovers. And -0.5 for each person who dies without infecting another person.

### (3d)

What are the actions and transition model associated with your MDP?

The actions for this MDP are based on the states, which are constantly changing. Typically, the actions for a virus in a given state are limited: do nothing, cause symptoms, or mutate. These are the options when one state is greater than 1 meter away from another state. However, when an occupied state is within one meter of one or more unoccupied states, it's actions are expanded; it can: do nothing, infect any one of the unoccupied states, cause symptoms, or mutate.

The transition model is different from the model in question one because it is time-dependent. With this in mind I am making my model such that a decision, or action, can be taken every second, for each individual virus. This means that each action has it's own transition probabilities that sum to one. The first case is a virus occupying a state that is too far away from another state to infect it. If it chooses to do nothing, it will do so with a probability of 0.99999834657, and accidentally cause symptoms with a probability of 0.00000165343. I found these numbers by taking 1/604800, one second out of a week. This is to ensure that the virus cannot lay dormant and undetectable in the people it infects as a strategy to infect everyone. Symptoms cause it to be noticed and perhaps eradicated. The next choice is to cause symptoms. The virus will want to do this if it wants to increase its ability to infect others or if it wants to terminate its current state at the end of the game. Causing symptoms will have a success rate of 1/3600, or once an hour if tried every second. This is to decrese the likelihood of incresing infection rate right before coming into contact with another person. if the probability was 1, the virus would simply wait until coming into contact with another person to show any symptoms. This makes it so the virus must "anticipate" when to develop symptoms in a given state. Finally we have the possibility of mutation. This will always be 1/0.03154e+9 and constantly executing simultaneously with other actions. This is once per 100 centuries, but if you take into account the number of viruses that will likely be present in the world, and the amount of time they may be present for, all it takes is one year for one mutation out of 10,000 viruses. Mutations will change the transition probabilities of the virus randomly such that they are either beneficial or detrimental to the virus.

For my model I am choosing to keep these probabilities the same when the state is close enough to another unoccupied state to infect it. This only leaves a transition model for the remaining action of infecting another person. When the virus is close enough to another person (1 meter), the action, "infect", is added to their list of available actions. This is a little tricky to do the math on. Let's say that if a person is touching another person, meaning the distance between them and the other person is 0, the probability of this person contracting the disease is 0.1. This would mean that 10 seconds of touching essentially guarentees another person of infection. We want this probability to approach 0 as the distance becomes closer to 1 meter. I am using an inverse square law to model this: probability of infection is $\frac{0.1}{D^2}$ where D is the distance between the infected state and the non-infected state, measured in millimeters. I think it is reasonable to say when two people are 0.01 meters apart they are essentially touching, and therefore at that distance the probability of infection is 0.1 per second. When people are 1 meter apart, the probability of infection is roughly 0.1 per 24 hours. The rate of infection failure is then of course 1 - $\frac{0.1}{D^2}$. Finally, there is the case where multiple people are within infection range. All that happens here is probability of infection is computed and executed if successful for each person within range, per second.

### (3e)

Interpret what an optimal policy represents in the context of your particular MDP.

An optimal policy is a decision function that chooses the best action for any given state, or one that chooses an action that maximizes utility or any given state. To be entirely honest, computing the optimal policy for this MDP is beyond me. There are a lot of factors to take into account, including the distribution of people on the planet, travel schedules, population densities, etc. But I can explain what the optimal policy represents and a rough outline of what it would include. The optimal policy would be one that results in the infection of every person on the planet, or the maximum amount possible (some people are probably completely solitary or in space or whatever), while minimizing the number of deaths without subsequent infection (because this results in loss of 0.5 points). The optimal policy would maximize the score of the virus. If you have ever played the game plague inc, an optimal policy for the first move is to create the first case in either greenland or madagascar, because those are the hardest placest to infect after an outbreak has started elsewhere. The optimal start policy for the virus would look something like that. The optimal policy for ANY given state would be to do nothing when not near anyone, and to infect when nearby unoccupied states/people. If it seems that two people's locations (one infected one not) are converging with a reasonable likelihood that they will cross paths, the virus should attempt to cause symptoms, because that increases the likelihood that the virus will infect the other person. The virus should never purposely choose to mutate because there is an equal chance of that hurting the virus's propagational abilities as it does benefiting the virus. In summary, when not near anybody: do nothing. When approaching an unoccupied state/person: cause symptoms. When within 1 meter of an unoccupied state: infect.

## Unit test

In [11]:
class Tests_Problem1(unittest.TestCase):
    def setUp(self):
        nrow = 3
        ncol = 3
        default_reward = -0.2
        discount = 0.5
        terminal = {(1,3):-1, (1,2):2}
        self.mdp = MDP(nrow, ncol, terminal, default_reward, discount)
    def test_actions_some(self):
        self.assertEqual(set(self.mdp.actions((2,2))) == {'N','S','E','W'}, True)
    def test_actions_few(self):
        self.assertEqual(set(self.mdp.actions((1,1))) == {'N','E'}, True)
    def test_actions_none(self):
        self.assertEqual(set(self.mdp.actions((1,2))) == {None}, True)
    def test_reward_t(self):
        self.assertEqual(self.mdp.reward((1,2)) == 2, True)
    def test_reward_nt(self):
        self.assertEqual(self.mdp.reward((2,2)) == -0.2, True)
    def test_result_nt(self):
        self.assertEqual(self.mdp.result((1,1), 'N') == (1,2), True)